In [2]:
import wget

In [4]:
keras_model_url = "https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5"
keras_model_path = "./models/bees-wasps.h5"

wget.download(keras_model_url, keras_model_path)

keras_model_path

100% [........................................................................] 89753864 / 89753864

'./models/bees-wasps.h5'

### Convert to tf-lite

In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model

keras_model = load_model(keras_model_path)

2023-11-28 15:58:52.009995: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

tflite_model = converter.convert()

tflite_model_path = "./models/bees-wasps.tflite"

with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

tflite_model_path

INFO:tensorflow:Assets written to: /var/folders/x3/fy30qts964b_nfn299fxgj5r0000gn/T/tmpg5o78bt3/assets


INFO:tensorflow:Assets written to: /var/folders/x3/fy30qts964b_nfn299fxgj5r0000gn/T/tmpg5o78bt3/assets
2023-11-28 16:00:20.900785: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-28 16:00:20.900809: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-28 16:00:20.902790: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/x3/fy30qts964b_nfn299fxgj5r0000gn/T/tmpg5o78bt3
2023-11-28 16:00:20.904235: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-28 16:00:20.904256: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/x3/fy30qts964b_nfn299fxgj5r0000gn/T/tmpg5o78bt3
2023-11-28 16:00:20.907724: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-28 16:00:20.908802: I tensorflow/cc/saved_model/load

'./models/bees-wasps.tflite'

In [7]:
!ls -lah models

total 286272
drwxr-xr-x  4 home  staff   128B Nov 28 16:00 .
drwxr-xr-x  5 home  staff   160B Nov 28 16:01 ..
-rw-r--r--  1 home  staff    86M Nov 28 15:53 bees-wasps.h5
-rw-r--r--  1 home  staff    43M Nov 28 16:00 bees-wasps.tflite


### Examine input/output indexes

In [8]:
# Step 4: Identify the output index for the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get the output details
output_details = interpreter.get_output_details()

# Display the output details
output_details


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

### Preprocessing

In [10]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size=(244, 244)):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [11]:
# Example usage:
image_url = "https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"
image = download_image(image_url)
prepared_image = prepare_image(image)

# Display the prepared image
prepared_image.show()


In [12]:
import numpy as np

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size=(150, 150)):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    img_array = np.array(img)  # Convert to Numpy array
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

# Example usage:
image_url = "https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"
image = download_image(image_url)
prepared_image = prepare_image(image)

# Display the value in the first pixel, R channel
r_channel_value = prepared_image[0, 0, 0]  # Assuming HWC format (height, width, channels)
print(f"The value in the first pixel, R channel: {r_channel_value}")


The value in the first pixel, R channel: 0.9450980392156862


Inference

In [13]:
# Assuming you have the model defined as described earlier in the conversation

# Load the model
model.load_weights('./models/bees-wasps.tflite')  # Replace with the actual path to your model weights

# Reshape the image to match the model's expected input shape
input_image = prepared_image.reshape(1, 150, 150, 3)

# Make the prediction
prediction = model.predict(input_image)

# Display the model's output
print(f"The model's output for the given image: {prediction}")


NameError: name 'model' is not defined

In [14]:
import tensorflow as tf

# Load the TensorFlow Lite model
tflite_model_path = "./models/bees-wasps.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Reshape the image to match the model's expected input shape
input_image = prepared_image.reshape(1, 150, 150, 3).astype('float32')

# Set the input tensor value
interpreter.set_tensor(input_details[0]['index'], input_image)

# Run inference
interpreter.invoke()

# Get the output tensor value
output = interpreter.get_tensor(output_details[0]['index'])

# Display the model's output
print(f"The model's output for the given image: {output}")


The model's output for the given image: [[0.6592137]]
